In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
class Deep_Narrow_ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(Deep_Narrow_ResNet, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
dn_model = Deep_Narrow_ResNet(BasicBlock, [13, 9, 3, 5])
num_params = sum(p.numel() for p in dn_model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")

Num Params: 4891338



In [ ]:
def train_val(model, criterion, optimizer, train_loader, val_loader, device, scheduler = None, use_scheduler = True):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)

        train_loss += loss.item()

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        loss.backward()

        optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

    train_accuracy = correct/total
    train_loss = train_loss/len(train_loader)
    
    model.eval()
    valid_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        valid_loss += cur_loss

    valid_accuracy = correct/total
    valid_loss = valid_loss/len(val_loader)
    
    if use_scheduler:
        scheduler.step(valid_accuracy)

    return train_loss, train_accuracy, valid_loss, valid_accuracy

def test(model, criterion, dataloader, device):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [ ]:
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wandb
import os
os.environ['WANDB_API_KEY'] = '072efef628ab06093f6625992e6a35809cd22154'
wandb.login()

wandb: Currently logged in as: ashwathb24 (nyu-tandon). Use `wandb login --relogin` to force relogin


True

In [ ]:
sweep_config = {
    "name": "hyperparameter_sweep",
    'metric': {'goal': 'maximize', 'name': 'test_accuracy'},
    "method": "random",
    "parameters": {

        "learning_rate": {
            "values": [0.01]
        },
        "scheduler": {
            "values": ["ReduceLROnPlateau"]
        },
        "optimizer":{
            "values":["sgd"]
        },
        "momentum": {
            "values": [0.7, 0.8, 0.9, 0.99]
        },
        "weight_decay": {
            "values": [1e-4, 5e-4, 1e-3, 5e-3]
        }
    }
}

In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'test_accuracy'},
 'name': 'hyperparameter_sweep',
 'parameters': {'learning_rate': {'values': [0.01]},
                'momentum': {'values': [0.7, 0.8, 0.9, 0.99]},
                'optimizer': {'values': ['sgd']},
                'scheduler': {'values': ['ReduceLROnPlateau']},
                'weight_decay': {'values': [0.0001, 0.0005, 0.001, 0.005]}}}


In [ ]:
def train_sweep():

  batch_size = 32
  wandb.init()
  config = wandb.config
  config.learning_rate = sweep_config['parameters']['learning_rate']
  config.optimizer = sweep_config['parameters']['optimizer']
  config.scheduler = sweep_config['parameters']['scheduler']
  config.momentum = sweep_config['parameters']['momentum']
  config.weight_decay = sweep_config['parameters']['weight_decay']


  transform_train = torchvision.transforms.Compose([
      torchvision.transforms.RandomCrop(32, padding=4),
      torchvision.transforms.RandomHorizontalFlip(),
      torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
      torchvision.transforms.ToTensor(),
      torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
  ])

  transform_test = torchvision.transforms.Compose([
                      torchvision.transforms.ToTensor(), 
                      torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

  trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

  # Split the train data into train and validation sets
  train_size = int(0.8 * len(trainset))
  val_size = len(trainset) - train_size
  trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

  testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

  train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
  val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
  test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

  classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  best_test_acc = 0

  torch.cuda.empty_cache()
  dn_model = Deep_Narrow_ResNet(BasicBlock, [13, 9, 3, 5])
  num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print(f"Num Params: {num_params}\n")
  model.to(device)
  criterion = nn.CrossEntropyLoss()
  if config.optimizer == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), config.learning_rate, momentum = config.momentum, weight_decay = config.weight_decay , nesterov=True)
  
  if config.scheduler == 'ReduceLROnPlateau':
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=0)

  train_losses_ = []
  train_accuracies_ = []
  valid_losses_ = []
  valid_accuracies_ = []

  epochs = 10

  for epoch in range(epochs):
      print(f"\n\tEpoch: {epoch}")

      train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                    train_loader, val_loader, device,
                                                                    scheduler = scheduler, use_scheduler = True)
      train_losses_.append(train_loss)
      train_accuracies_.append(train_accuracy)
      valid_losses_.append(val_loss)
      valid_accuracies_.append(val_accuracy)
      print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
      print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

  test_loss, test_accuracy = test(model, criterion, test_loader, device)
  print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

  if test_accuracy > best_test_acc:
      best_test_acc = test_accuracy

      torch.save(model.state_dict(), 'zigzag_resnet_7blocks.pth')

      metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                      'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                      'test_loss': test_loss, 'test_accuracy': test_accuracy}
  wandb.log({'test_accuracy': test_accuracy})
  wandb.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = "Modified-Resnet")

Create sweep with ID: dqtdxs3j
Sweep URL: https://wandb.ai/nyu-tandon/Modified-Resnet/sweeps/dqtdxs3j


In [ ]:
wandb.agent(sweep_id,train_sweep, count = 16)

wandb: Agent Starting Run: o4iorbdu with config:
wandb: 	learning_rate: 0.01
wandb: 	momentum: 0.8
wandb: 	optimizer: sgd
wandb: 	scheduler: ReduceLROnPlateau
wandb: 	weight_decay: 0.0005


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'optimizer' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'scheduler' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'momentum' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


100%|██████████| 170498071/170498071 [00:13<00:00, 13102397.91it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Num Params: 4891338


	Epoch: 0
	Training Loss: 3.5343; Training Accuracy: 34.63%
	Validation Loss: 1.5133; Validation Accuracy: 43.81%

	Epoch: 1
	Training Loss: 2.802; Training Accuracy: 49.3525%
	Validation Loss: 1.2686; Validation Accuracy: 53.65%

	Epoch: 2
	Training Loss: 2.2898; Training Accuracy: 59.355%
	Validation Loss: 1.1099; Validation Accuracy: 60.75%

	Epoch: 3
	Training Loss: 1.914; Training Accuracy: 66.49%
	Validation Loss: 0.9072; Validation Accuracy: 68.39%

	Epoch: 4
	Training Loss: 1.6823; Training Accuracy: 70.93%
	Validation Loss: 0.8148; Validation Accuracy: 71.32%

	Epoch: 5
	Training Loss: 1.5205; Training Accuracy: 73.5675%
	Validation Loss: 0.7617; Validation Accuracy: 73.64%

	Epoch: 6
	Training Loss: 1.3922; Training Accuracy: 76.09%
	Validation Loss: 0.6937; Validation Accuracy: 75.6%

	Epoch: 7
	Training Loss: 1.2924; Training Accuracy: 77.9375%
	Validation Loss: 0.

test_accuracy,▁
test_accuracy,0.8132


...

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338


	Epoch: 0
	Training Loss: 3.6377; Training Accuracy: 32.785%
	Validation Loss: 1.5984; Validation Accuracy: 40.95%

	Epoch: 1
	Training Loss: 3.025; Training Accuracy: 44.64%
	Validation Loss: 1.3501; Validation Accuracy: 50.8%

	Epoch: 2
	Training Loss: 2.6078; Training Accuracy: 53.195%
	Validation Loss: 1.115; Validation Accuracy: 59.65%

	Epoch: 3
	Training Loss: 2.1957; Training Accuracy: 60.7725%
	Validation Loss: 0.968; Validation Accuracy: 65.8%

	Epoch: 4
	Training Loss: 1.9261; Training Accuracy: 66.0075%
	Validation Loss: 0.9734; Validation Accuracy: 65.8%

	Epoch: 5
	Training Loss: 1.5543; Training Accuracy: 72.8725%
	Validation Loss: 0.7589; Validation Accuracy: 73.2%

	Epoch: 6
	Training Loss: 1.4428; Training Accuracy: 75.0125%
	Validation Loss: 0.7193; Validation Accuracy: 74.51%

	Epoch: 7
	Training Loss: 1.3564; Training Accuracy: 76.52%
	Validation Loss: 0.6794; Validati

test_accuracy,▁
test_accuracy,0.7919


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: qieie64z with config:
wandb: 	learning_rate: 0.01
wandb: 	momentum: 0.7
wandb: 	optimizer: sgd
wandb: 	scheduler: ReduceLROnPlateau
wandb: 	weight_decay: 0.005


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'optimizer' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'scheduler' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'momentum' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338


	Epoch: 0
	Training Loss: 3.4475; Training Accuracy: 36.35%
	Validation Loss: 1.5173; Validation Accuracy: 44.78%

	Epoch: 1
	Training Loss: 2.7286; Training Accuracy: 51.1775%
	Validation Loss: 1.2208; Validation Accuracy: 55.29%

	Epoch: 2
	Training Loss: 2.2257; Training Accuracy: 61.255%
	Validation Loss: 1.1033; Validation Accuracy: 61.01%

	Epoch: 3
	Training Loss: 1.9425; Training Accuracy: 66.52%
	Validation Loss: 1.0329; Validation Accuracy: 64.8%

	Epoch: 4
	Training Loss: 1.7813; Training Accuracy: 69.5175%
	Validation Loss: 0.956; Validation Accuracy: 67.71%

	Epoch: 5
	Training Loss: 1.679; Training Accuracy: 71.6475%
	Validation Loss: 0.8442; Validation Accuracy: 71.31%

	Epoch: 6
	Training Loss: 1.5684; Training Accuracy: 73.325%
	Validation Loss: 0.7976; Validation Accuracy: 72.93%

	Epoch: 7
